In [1]:
!pip install pandas_ta
!pip install backtesting
import datetime
import pandas_ta as ta
import pandas as pd
import matplotlib.pyplot as plt
from backtesting import Backtest
from backtesting import Strategy
from backtesting.lib import crossover
import yfinance as yf
import seaborn as sns

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.1/115.1 kB 1.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pandas_ta: filename=pandas_ta-0.3.14b0-py3-none-any.whl size=218907 sha256=6245da32af51c76e15f9103b64c835aa7432e75cd6da7b1692ff2be04925a6fb
  Stored in directory: /root/.cache/pip/wheels/69/00/ac/f7fa862c34b0e2ef320175100c233377b4c558944f12474cf0
Successfully built pandas_ta
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 175.5/175.5 kB 3.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for backtesting: filename=Backtesting-0.3.3-py3-none-any.whl size=173916 sha256=fe7e4adb5dfe82d65f1ece866df725e0f6c303261989b072c25a0747f6d42017
  Stored in directory: /root/.cache/pip/wheels/e2/30/7f/19cbe31987c6ebdb47f1f510343249066711609e3da2d57176
Successfully built backtesting


/usr/local/lib/python3.10/dist-packages/backtesting/_plotting.py:50: UserWarning: Jupyter Notebook detected. Setting Bokeh output to notebook. This may not work in Jupyter clients without JavaScript support (e.g. PyCharm, Spyder IDE). Reset with `backtesting.set_bokeh_output(notebook=False)`.
  warnings.warn('Jupyter Notebook detected. '
/usr/local/lib/python3.10/dist-packages/yfinance/base.py:48: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  _empty_series = pd.Series()


In [ ]:
class SMABasedStrategy(Strategy):
    sma_1_l = 14

    def init(self):
        self.sma_1 = self.I(ta.sma, pd.Series(self.data.Close), self.sma_1_l)


    def next(self):
        price = self.data.Close[-1]
        if  self.sma_1[-1] < price :
            self.buy()
        elif self.sma_1[-1] > price:
            self.position.close()

start = '2020-01-01'
end = '2024-01-01'
interval = "1d"
tickers = ["SPY", "AAPL", "GOOG", "CVX", "HMY", "NVDA", "AMD"]
tickers = ["SPY"]

for ticker in tickers:
  df = download_data(ticker, start, end, interval)
  bt = Backtest(df, SMABasedStrategy, cash=10_000, commission=.002)

  stats = bt.optimize(
          sma_1_l = range(1,200,5),
          maximize='Return [%]')

  #stats = bt.run()
  bt.plot()
  print(stats["Sharpe Ratio"])
  print(stats["Return [%]"])
  print(stats["_strategy"])
  print(stats["# Trades"])
  print(stats["Avg. Trade Duration"])

[*********************100%%**********************]  1 of 1 completed


Backtest.optimize:   0%|          | 0/2 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/backtesting/_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%d %b'
  formatter=DatetimeTickFormatter(days=['%d %b', '%a %d'],
/usr/local/lib/python3.10/dist-packages/backtesting/_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%m/%Y'
  formatter=DatetimeTickFormatter(days=['%d %b', '%a %d'],
/usr/local/lib/python3.10/dist-packages/backtesting/_plotting.py:659: UserWarning: found multiple competing values for 'toolbar.active_drag' property; using the latest value
  fig = gridplot(
/usr/local/lib/python3.10/dist-packages/backtesting/_plotting.py:659: UserWarning: found multiple competing values for 'toolbar.active_scroll' property; using the latest value
  fig = gridplot(


In [2]:
def download_data(ticker, start, end, interval):
  data = yf.download(ticker, start=start, end=end, interval = interval)
  df = pd.DataFrame(data)
  df.drop("Adj Close", axis=1, inplace=True)
  return df

In [ ]:
class SMABasedStrategy(Strategy):
    sma_1_l = 14
    sma_2_l = 21
    sma_3_l = 35
    def init(self):
        self.sma_1 = self.I(ta.sma, pd.Series(self.data.Close), self.sma_1_l)
        self.sma_2 = self.I(ta.sma, pd.Series(self.data.Close), self.sma_2_l)
        self.sma_3 = self.I(ta.sma, pd.Series(self.data.Close), self.sma_3_l)

    def are_smas_upwards(self):
        return self.sma_1[-1] > self.sma_1[-2]  and self.sma_2[-1] < self.sma_2[-2] and self.sma_3[-1] < self.sma_3[-2]
    def are_smas_upwards2(self):
        return self.sma_1[-1] > self.sma_1[-2]  and self.sma_2[-1] > self.sma_2[-2] and self.sma_3[-1] > self.sma_3[-2]
    def are_smas_downwards(self):
        return self.sma_1[-1] < self.sma_1[-2]  and self.sma_2[-1] > self.sma_2[-2] and self.sma_3[-1] > self.sma_3[-2]

    def next(self):
        price = self.data.Close[-1]
        if self.are_smas_upwards() and self.sma_3[-1] < price :
            self.buy(tp=1.15*price, sl=0.95*price)
        elif self.are_smas_upwards2() and self.sma_3[-1] < price :
            self.buy(tp=1.15*price, sl=0.95*price)
        elif self.are_smas_downwards():
            self.position.close()

start = '2020-01-01'
end = '2024-01-01'
interval = "1d"
tickers = ["SPY", "AAPL", "GOOG", "CVX", "HMY", "NVDA", "AMD"]
#tickers = ["SPY"]

for ticker in tickers:
  df = download_data(ticker, start, end, interval)
  bt = Backtest(df, SMABasedStrategy, cash=10_000, commission=.002)

  stats = bt.optimize(
          sma_1_l = range(1,20,5),
          sma_2_l = range(20,50,5),
          sma_3_l = range(50,100,5),
          maximize='Return [%]')

  #stats = bt.run()
  bt.plot()
  print(stats)
  print(stats["Sharpe Ratio"])
  print(stats["Return [%]"])
  print(stats["_strategy"])
  print(stats["# Trades"])
  print(stats["Avg. Trade Duration"])

[*********************100%%**********************]  1 of 1 completed


Backtest.optimize:   0%|          | 0/2 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/backtesting/_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%d %b'
  formatter=DatetimeTickFormatter(days=['%d %b', '%a %d'],
/usr/local/lib/python3.10/dist-packages/backtesting/_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%m/%Y'
  formatter=DatetimeTickFormatter(days=['%d %b', '%a %d'],
/usr/local/lib/python3.10/dist-packages/backtesting/_plotting.py:659: UserWarning: found multiple competing values for 'toolbar.active_drag' property; using the latest value
  fig = gridplot(
/usr/local/lib/python3.10/dist-packages/backtesting/_plotting.py:659: UserWarning: found multiple competing values for 'toolbar.active_scroll' property; using the latest value
  fig = gridplot(


Start                     2020-01-02 00:00:00
End                       2023-12-29 00:00:00
Duration                   1457 days 00:00:00
Exposure Time [%]                   49.502982
Equity Final [$]                 14989.472052
Equity Peak [$]                  15061.794583
Return [%]                          49.894721
Buy & Hold Return [%]               46.307755
Return (Ann.) [%]                    10.67103
Volatility (Ann.) [%]                9.279661
Sharpe Ratio                         1.149938
Sortino Ratio                        1.888565
Calmar Ratio                         1.592994
Max. Drawdown [%]                   -6.698727
Avg. Drawdown [%]                   -1.536015
Max. Drawdown Duration      324 days 00:00:00
Avg. Drawdown Duration       28 days 00:00:00
# Trades                                   27
Win Rate [%]                         70.37037
Best Trade [%]                      14.947907
Worst Trade [%]                     -3.576165
Avg. Trade [%]                    

[*********************100%%**********************]  1 of 1 completed


Backtest.optimize:   0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
class SMABasedStrategy(Strategy):
    sma_1_l = 14
    sma_2_l = 21

    def init(self):
        self.sma_1 = self.I(ta.sma, pd.Series(self.data.Close), self.sma_1_l)
        self.sma_2 = self.I(ta.sma, pd.Series(self.data.Close), self.sma_2_l)
    def are_smas_upwards(self):
        return self.sma_1[-1] > self.sma_1[-2]  and self.sma_2[-1] < self.sma_2[-2]
    def are_smas_downwards(self):
        return self.sma_1[-1] < self.sma_1[-2]  and self.sma_2[-1] > self.sma_2[-2]
    def next(self):
        price = self.data.Close[-1]
        low = self.data.Low[-1]
        if self.are_smas_upwards() :
            self.buy()
            #self.buy(tp=1.05*price, sl=0.95*price)
        elif self.are_smas_downwards() :
            self.position.close()
start = '2010-01-01'
end = '2023-12-31'
tickers = ["SPY", "AAPL", "GOOG", "CVX", "HMY", "NVDA", "AMD"]
tickers = ["SPY"]
interval = "1d"
for ticker in tickers:
  df = download_data(ticker, start, end, interval)

  bt = Backtest(df, SMABasedStrategy, cash=10_000, commission=.002)
  stats = bt.optimize(
          sma_1_l = range(5,25,5),
          sma_2_l = range(25,55,5),
          maximize='Return [%]')
  #stats = bt.run()
  bt.plot()
  print(stats["Sharpe Ratio"])
  print(stats["Return [%]"])
  print(stats["_strategy"])
  print(stats["# Trades"])
  print(stats["Avg. Trade Duration"])

[*********************100%%**********************]  1 of 1 completed


Backtest.optimize:   0%|          | 0/2 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/backtesting/_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%d %b'
  formatter=DatetimeTickFormatter(days=['%d %b', '%a %d'],
/usr/local/lib/python3.10/dist-packages/backtesting/_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%m/%Y'
  formatter=DatetimeTickFormatter(days=['%d %b', '%a %d'],
/usr/local/lib/python3.10/dist-packages/backtesting/_plotting.py:659: UserWarning: found multiple competing values for 'toolbar.active_drag' property; using the latest value
  fig = gridplot(
/usr/local/lib/python3.10/dist-packages/backtesting/_plotting.py:659: UserWarning: found multiple competing values for 'toolbar.active_scroll' property; using the latest value
  fig = gridplot(


0.643074692328635
151.0751673905944
SMABasedStrategy(sma_1_l=10,sma_2_l=35)
43
41 days 00:00:00


In [4]:
class RsiOscillator(Strategy):
    upper_bound = 80
    lower_bound = 20
    rsi_window = 3

    def init(self):
        self.rsi = self.I(ta.rsi, pd.Series(self.data.Close), self.rsi_window)

    def next(self):
        price = self.data.Close[-1]
        if self.rsi[-1] > self.upper_bound :
            self.position.close()
        #elif self.rsi[-3] < self.lower_bound and self.rsi[-3] > self.rsi[-2] and self.rsi[-2] < self.rsi[-1]:
        elif self.rsi[-1] < self.lower_bound  :
            self.buy(tp=1.05*price, sl=0.95*price)
            #self.buy()


start = '2020-01-01'
end = '2023-12-31'
tickers = ["SPY","MMM","ADBE", "AEM", "AMGN", "AAPL", "BAC", "GOLD", "BIOX", "GOOG", "CVX", "LLY", "XOM", "FSLR" , "IBM" , "JD", "JPM" , "HMY","MELI", "NFLX" , "PEP", "PFE", "PG", "RIO", "SONY", "SBUX", "BA","TM","VZ","VIST","WMT", "NVDA", "AMD"]
tickers = ["SPY"]
interval = "1d"
for ticker in tickers:
  df = download_data(ticker, start, end, interval)
  bt = Backtest(df, RsiOscillator, cash=10_000, commission=.002)

  stats = bt.optimize(
          upper_bound = range(45,95,5),
          lower_bound = range(5,55,5),
          rsi_window = range(3,21,1),
          maximize="Sharpe Ratio")

  #stats = bt.run()

  bt.plot()

  print(stats)

  if float(stats["Sharpe Ratio"]) > 0.8 and float(stats["# Trades"]) > 50 or True:
    print("")
    print(ticker)
    print(stats["Sharpe Ratio"])
    print(stats["Return [%]"])
    print(stats["_strategy"])
    print(stats["# Trades"])
    print(stats["Avg. Trade Duration"])


[*********************100%%**********************]  1 of 1 completed
/usr/local/lib/python3.10/dist-packages/backtesting/backtesting.py:1488: UserWarning: Searching for best of 1800 configurations.
  output = _optimize_grid()


Backtest.optimize:   0%|          | 0/6 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/backtesting/_stats.py:122: RuntimeWarning: divide by zero encountered in double_scalars
  s.loc['Sortino Ratio'] = np.clip((annualized_return - risk_free_rate) / (np.sqrt(np.mean(day_returns.clip(-np.inf, 0)**2)) * np.sqrt(annual_trading_days)), 0, np.inf)  # noqa: E501
/usr/local/lib/python3.10/dist-packages/backtesting/_stats.py:122: RuntimeWarning: divide by zero encountered in double_scalars
  s.loc['Sortino Ratio'] = np.clip((annualized_return - risk_free_rate) / (np.sqrt(np.mean(day_returns.clip(-np.inf, 0)**2)) * np.sqrt(annual_trading_days)), 0, np.inf)  # noqa: E501
/usr/local/lib/python3.10/dist-packages/backtesting/_stats.py:122: RuntimeWarning: divide by zero encountered in double_scalars
  s.loc['Sortino Ratio'] = np.clip((annualized_return - risk_free_rate) / (np.sqrt(np.mean(day_returns.clip(-np.inf, 0)**2)) * np.sqrt(annual_trading_days)), 0, np.inf)  # noqa: E501
/usr/local/lib/python3.10/dist-packages/backtesting/_stats.py:122: 

Start                     2020-01-02 00:00:00
End                       2023-12-29 00:00:00
Duration                   1457 days 00:00:00
Exposure Time [%]                    0.795229
Equity Final [$]                 11346.609904
Equity Peak [$]                  11346.609904
Return [%]                          13.466099
Buy & Hold Return [%]               46.307755
Return (Ann.) [%]                    3.215234
Volatility (Ann.) [%]                2.972076
Sharpe Ratio                         1.081814
Sortino Ratio                       32.402317
Calmar Ratio                        16.225321
Max. Drawdown [%]                   -0.198161
Avg. Drawdown [%]                   -0.198161
Max. Drawdown Duration      687 days 00:00:00
Avg. Drawdown Duration      687 days 00:00:00
# Trades                                    4
Win Rate [%]                             75.0
Best Trade [%]                       7.988214
Worst Trade [%]                     -0.199601
Avg. Trade [%]                    